In [ ]:
# Download and extract the dataset from CMS

import requests # to download the dataset
import zipfile # to extract from archive
import shutil # to write the dataset to file
import os # rename file to something more type-able
import pandas as pd
import numpy as np

data_dir = '../data/'

url = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/Downloads/Part_D_All_Drugs_2015.zip'
response = requests.get(url, stream=True)

with open(data_dir + 'dataset.zip', 'wb') as ds_zipout:
    shutil.copyfileobj(response.raw, ds_zipout)

zip = zipfile.ZipFile(data_dir + 'dataset.zip', 'r')
ds_filename = zip.namelist()[0]
zip.extract(ds_filename, path=data_dir)

We're going to read the data into a `pandas.DataFrame`, but because the data is in several work sheets, we need to read out the right work sheet:

In [ ]:
xls = pd.ExcelFile("../data/Medicare_Drug_Spending_PartD_All_Drugs_YTD_2015_12_06_2016.xlsx")
df = xls.parse('Data', skiprows=3)
df.index = np.arange(1, len(df) + 1)


In [ ]:
# Capture only the drug names (we'll need this later)
df_drugnames = df.iloc[:, :2]
df_drugnames.columns = [
    'drugname_brand',
    'drugname_generic'
]

In [ ]:
# Strip extraneous whitespace from drug names
df_drugnames.loc[:, 'drugname_brand'] = df_drugnames.loc[:, 'drugname_brand'].map(lambda x: x.strip())
df_drugnames.loc[:, 'drugname_generic'] = df_drugnames.loc[:, 'drugname_generic'].map(lambda x: x.strip())

In [ ]:
# Serialize drug names to feather file for use in both Python and R
import feather
feather.write_dataframe(df_drugnames, data_dir + 'drugnames.feather')

In [ ]:
    # Separate column groups by year
    cols_by_year = [
        { 'year': 2011, 'start': 2, 'end': 12 },
        { 'year': 2012, 'start': 12, 'end': 22 },
        { 'year': 2013, 'start': 22, 'end': 32 },
        { 'year': 2014, 'start': 32, 'end': 42 },
        { 'year': 2015, 'start': 42, 'end': 53 },
    ]

    df_years = {}

    col_brandname = 0
    col_genericname = 1
    for cols in cols_by_year:
        year, start, end = cols['year'], cols['start'], cols['end']

        df_years[year] = pd.concat(
            [
                df_drugnames,
                df.iloc[:, start:end]
            ],
            axis=1)

In [ ]:
# Remove 2015's extra column for "Annual Change in Average Cost Per Unit" (we can calculate it, anyhow)
df_years[2015] = df_years[2015].drop(df_years[2015].columns[-1], axis=1)

In [ ]:
# Drop any rows in each year that have absolutely no data, then reset their row indices
for year in df_years:
    nonnull_rows = df_years[year].iloc[:, 2:].apply(lambda x: x.notnull().any(), axis=1)
    df_years[year] = df_years[year][nonnull_rows]
    df_years[year].index = np.arange(1, len(df_years[year]) + 1)

In [ ]:
# Make columns easier to type and more generic w.r.t. year
generic_columns = [
    "drugname_brand",
    "drugname_generic",
    "claim_count",
    "total_spending",
    "user_count",
    "total_spending_per_user",
    "unit_count",
    "unit_cost_wavg",
    "user_count_non_lowincome",
    "out_of_pocket_avg_non_lowincome",
    "user_count_lowincome",
    "out_of_pocket_avg_lowincome"
]

for year in df_years:
    df_years[year].columns = generic_columns

In [ ]:
# Cast all column data to appropriate numeric types

# Suppress SettingWithCopyWarnings because I think it's
# tripping on the fact that we have a dict of DataFrames
pd.options.mode.chained_assignment = None
for year in df_years:
    # Ignore the first two columns, which are strings and contain drug names
    for col in df_years[year].columns[2:]:
        df_years[year].loc[:, col] = pd.to_numeric(df_years[year][col])
pd.options.mode.chained_assignment = 'warn'

In [ ]:
# Serialize data for each year to feather file for use in both Python and R
for year in df_years:
    feather.write_dataframe(df_years[year], data_dir + 'spending-' + str(year) + '.feather')